In [ ]:
!pip install "pyiceberg[s3fs,hive]"

In [77]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [ ]:
#i create a catalog_namespace for hive to access minio using hive with nameSpace
catalog.create_namespace("docs_raj")

In [30]:
#list name_space with acces hive with minio
catalog.list_namespaces()

[('default',),
 ('docs_raj',),
 ('hi',),
 ('hlo',),
 ('pod',),
 ('power_pod',),
 ('powerlevelpoddata',),
 ('powerpod',),
 ('saravana',),
 ('saro',),
 ('sh',),
 ('sharanya',),
 ('sk',),
 ('soh',)]

In [76]:
#list the namespace
catalog.list_tables("docs_raj")

[('docs_raj', 'bids')]

In [75]:
#drop the create the namespace
catalog.drop_table("docs_raj.demo")

In [84]:
#create a shemma for the table
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, DoubleType

schema = Schema(
    NestedField(1, "city", StringType(), required=False),
    NestedField(2, "lat", DoubleType(), required=False),
    NestedField(3, "long", DoubleType(), required=False),
)

tbl = catalog.create_table(
    identifier="docs_raj.rajjjj", #mendtion identifier for namespace and create a table name
    schema=schema ,
    location="s3a://mari/raj_data" #mention the  minio bucket name and mendtion path name
)


In [79]:
#push the data mention the table
import pyarrow as pa

df = pa.Table.from_pylist(
    [
        {"city": "Amsterdam", "lat": 52.371807, "long": 4.896029},
        {"city": "San Francisco", "lat": 37.773972, "long": -122.431297},
        {"city": "Drachten", "lat": 53.11254, "long": 6.0989},
        {"city": "Paris", "lat": 48.864716, "long": 2.349014},
    ],
)

tbl.append(df)

In [80]:
tbl.scan().to_arrow()

pyarrow.Table
city: string
lat: double
long: double
----
city: [["Amsterdam","San Francisco","Drachten","Paris"]]
lat: [[52.371807,37.773972,53.11254,48.864716]]
long: [[4.896029,-122.431297,6.0989,2.349014]]

In [81]:
table = catalog.load_table("docs_raj.bids")

In [82]:
table.scan().to_arrow()

pyarrow.Table
city: string
lat: double
long: double
----
city: [["Amsterdam","San Francisco","Drachten","Paris"]]
lat: [[52.371807,37.773972,53.11254,48.864716]]
long: [[4.896029,-122.431297,6.0989,2.349014]]